In [28]:
import re, glob, nltk
from collections import Counter
from math import log
from math import sqrt

In [29]:
# global variable
idx_table = {}
docu_table = {}
weight_table = {}
stop_words = set()
f = open('stops.txt', 'r')
for line in f:
    stop_words.add(line.strip())
f.close()
retrieval_model = 'lm'

In [30]:
def df(term):
    return len(idx_table[term])

def idf(term):
    return log(len(docu_table) / df(term), 10)

def cosineSimilarity(query, docno, query_counter):
    c_dividend = 0 # dividend in formula
    c_divisor_d = 0 # document divisor in formula
    c_divisor_w = 0 # query divisor in formula
    for term in query:
        w = query_counter[term] * idf(term)
        d = docu_table[docno][term] * idf(term)
        c_dividend += w * d
        c_divisor_w += w ** 2
    for term in docu_table[docno]:
        d = docu_table[docno][term] * idf(term)
        c_divisor_d += d ** 2
    return c_dividend / sqrt(c_divisor_d * c_divisor_w)

def bm25Similarity(query, docno, query_counter, totaldl):
    k1 = 1.2
    k2 = 500
    b = 0.75
    N = len(docu_table)
    D = sum(docu_table[docno].values())
    avgdl = totaldl / len(docu_table)
    res = 0
    for term in query:
        n = len(idx_table[term])
        tf = idx_table[term][docno]
        p1 = (k1 + 1) * tf / (tf + k1 * (1 - b + b * D / avgdl))
        p2 = (k2 + 1) * query_counter[term] / (k2 + query_counter[term])
        res += idf(term) * p1 * p2
    return res

def lmSimilarity(query, docno, query_counter, totaldl):
    D = sum(docu_table[docno].values())
    avgdl = totaldl / len(docu_table)
    res = 0
    for term in query:
        tf = idx_table[term][docno]
        tfC = sum(idx_table[term].values())
        res += log(((tf + avgdl * tfC / totaldl) / (D + avgdl)), 10)
    return res

def relevanceRanking(query, totaldl):
    result = Counter()
    # find relevant doc containing at least one query term
    relevant_doc = set()
    for term in query:
        for docno in idx_table[term]:
            relevant_doc.add(docno)
    # build query counter to calculate tf in query
    query_counter = Counter()
    for term in query:
        query_counter[term] += 1
    # calculate relevance
    for docno in relevant_doc:
        if retrieval_model == 'cosine':
            result[docno] = cosineSimilarity(query, docno, query_counter)
        elif retrieval_model == 'bm25':
            result[docno] = bm25Similarity(query, docno, query_counter, totaldl)
        elif retrieval_model == 'lm':
            result[docno] = lmSimilarity(query, docno, query_counter, totaldl)
    return result.most_common(20)

In [31]:
def prefixReplace(match):
    prefix, stem = match.group(1), match.group(2)
    temp = prefix + stem
    if not stem in stop_words:
        temp += ' ' + stem
    return temp

def hyphenReplace(match):
    temp = match.group()
    li = temp.split('-')
    temp = temp.replace('-', '')
    for item in li:
        if not item in stop_words:
            temp += ' ' + item
    return temp

def loadIndexTables():
    f = open('single_term_idx_table.txt', 'r')
    totaldl = 0
    for line in f:
        parts = line.strip().split()
        if not parts[0] in idx_table:
            idx_table[parts[0]] = Counter()
        if not parts[1] in docu_table:
            docu_table[parts[1]] = Counter()
        # term, docno, term frequency
        idx_table[parts[0]][parts[1]] = int(parts[2])
        docu_table[parts[1]][parts[0]] = int(parts[2])
        totaldl += int(parts[2])
    f.close()
    # calculate tf*idf and store in cluster table
    for docno in docu_table:
        weight_table[docno] = Counter()
        for term in docu_table[docno]:
            weight_table[docno][term] = docu_table[docno][term] * idf(term)
    return totaldl

def queryOutput(query_path, output_file, totaldl):
    if query_path[0] == '/':
        query_path = '.' + query_path
    if query_path[-1] != '/':
        query_path += '/'
    
    files = glob.glob(query_path+'*') # grab all the files under path
    output_file = open(output_file, 'w') # open output file
    output_file.write('Descritpion_ID,Top_20_Image_IDs\n')
    p_prefix = re.compile(r'\b(a|an|ante|anti|auto|circum|co|com|con|contra|contro|de|dis|en|em|ex|extra|fore|hetero|homo|homeo|hyper|il|im|in|ir|inter|intra|intro|macro|micro|mid|mis|mono|non|omni|over|post|pre|pro|re|semi|sub|super|sym|syn|trans|tri|un|under|uni)-([a-z])+\b', re.I)
    p_hyphen = re.compile(r'\b(\w+-)+\w+\b')
    get_num = re.compile(r'(\d+)')
    for file in files:
        f = open(file, 'r')
        docno = int(get_num.search(file).group(1))
        output_file.write(str(docno)+".txt,")
        query = []
        for line in f:
            line = line.strip()
            line = line.lower()
            # expand stem with hyphen prefix
            line = p_prefix.sub(prefixReplace, line)
            # expand hyphenated word
            line = p_hyphen.sub(hyphenReplace, line)
            line = line.replace(':', ' ')
            line = nltk.word_tokenize(line)
            query += [x for x in line if x in idx_table]
        result = relevanceRanking(query, totaldl)
        # print(result)
        output_file.write(' '.join(x[0]+'.jpg' for x in result))
        output_file.write('\n')
    output_file.close()

totaldl = loadIndexTables()
queryOutput('data/descriptions_test', retrieval_model+'submission.csv', totaldl)